In [9]:
import pandas as pd
import numpy as np
import re

# 지난 시간 연습 문제 풀이

In [2]:
train = pd.read_csv("data/titanic/train.csv")
test = pd.read_csv("data/titanic/test.csv")
train_test_data = [train, test]
train_test_data

[     PassengerId  Survived  Pclass  \
 0              1         0       3   
 1              2         1       1   
 2              3         1       3   
 3              4         1       1   
 4              5         0       3   
 5              6         0       3   
 6              7         0       1   
 7              8         0       3   
 8              9         1       3   
 9             10         1       2   
 10            11         1       3   
 11            12         1       1   
 12            13         0       3   
 13            14         0       3   
 14            15         0       3   
 15            16         1       2   
 16            17         0       3   
 17            18         1       2   
 18            19         0       3   
 19            20         1       3   
 20            21         0       2   
 21            22         1       2   
 22            23         1       3   
 23            24         1       1   
 24            25        

정규표현식 사용 -> 호칭 추출 -> 호칭별 건수 출력

In [5]:
for data in train_test_data:
    data["Title"] = data["Name"].str.extract("\, (?P<title>[a-zA-Z]+)\.")

Mr: 0, Miss: 1, Mrs: 2, 나머지는 모두 3으로 처리 -> 별도의 컬럼을 추가해서 저장

In [8]:
mapping={"Mr":0, "Miss":1,"Mrs":2}
for data in train_test_data:
    data["titleno"] = data['Title'].map(mapping)
train_test_data

[     PassengerId  Survived  Pclass  \
 0              1         0       3   
 1              2         1       1   
 2              3         1       3   
 3              4         1       1   
 4              5         0       3   
 5              6         0       3   
 6              7         0       1   
 7              8         0       3   
 8              9         1       3   
 9             10         1       2   
 10            11         1       3   
 11            12         1       1   
 12            13         0       3   
 13            14         0       3   
 14            15         0       3   
 15            16         1       2   
 16            17         0       3   
 17            18         1       2   
 18            19         0       3   
 19            20         1       3   
 20            21         0       2   
 21            22         1       2   
 22            23         1       3   
 23            24         1       1   
 24            25        

# 교차 검증

k-fold cross-validation (k겹 교차검증): 데이터를 k번씩 1/k로 나누고 모델을 학습. 보통 5 또는 10 사용.

("파이썬 라이브러리를 활용한 머신러닝" 324쪽 참고)

# 오차 행렬

끝단어가 예측값이라고 외우면 쉽다
* True Positive: 양성으로 예측 (맞은 예측)
* False Positive: 양성으로 예측 (틀린 예측)
* True Negative: 음성으로 예측 (맞은 예측)
* False Positive: 음성으로 예측 (틀린 예측)

("파이썬 라이브러리를 활용한 머신러닝" 362쪽 참고)

# 데이터 전처리

In [10]:
train = pd.read_csv("data/titanic/train.csv")
test = pd.read_csv("data/titanic/test.csv")

In [13]:
# 결측값을 평균으로 대체
train["Age_mean"] = train["Age"]
train["Age_mean"].fillna(train["Age_mean"].mean(), inplace=True)
test["Age_mean"] = test["Age"]
test["Age_mean"].fillna(test["Age_mean"].mean(), inplace=True)

In [14]:
train["Age_mean"].isnull().sum()

0

In [17]:
train["Age"].mean()    # 29.69
train["Age_mean"].mean()    # 29.69

29.699117647058763

In [18]:
train["Sex"].value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [19]:
train["Gender"] = (train["Sex"]=="female")    # female이면 True, male이면 False 값이 들어간다
test["Gender"] = (test["Sex"]=="female")

In [20]:
train["Embarked"].value_counts()    # value_counts로는 NaN의 존재는 확인할 수 없다

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [21]:
train["Embarked"].isnull().sum()

2

In [22]:
test["Embarked"].isnull().sum()

0

In [23]:
train["Embarked"]
# 원핫 인코딩 --> S:100, C:010, Q:001

0      S
1      C
2      S
3      S
4      S
5      Q
6      S
7      S
8      S
9      C
10     S
11     S
12     S
13     S
14     S
15     S
16     Q
17     S
18     S
19     C
20     S
21     S
22     Q
23     S
24     S
25     S
26     C
27     S
28     Q
29     S
      ..
861    S
862    S
863    S
864    S
865    S
866    C
867    S
868    S
869    S
870    S
871    S
872    S
873    S
874    C
875    C
876    S
877    S
878    S
879    C
880    S
881    S
882    S
883    S
884    S
885    Q
886    S
887    S
888    S
889    C
890    Q
Name: Embarked, Length: 891, dtype: object

## (다른 방법의) 원핫 인코딩

In [24]:
train["Embarked_S"] = (train["Embarked"]=="S")
train["Embarked_C"] = (train["Embarked"]=="C")
train["Embarked_Q"] = (train["Embarked"]=="Q")

In [26]:
train["Embarked"].value_counts()
train["Embarked_S"].sum()

644

In [27]:
train[["Embarked_S", "Embarked_C", "Embarked_Q"]]

,Embarked_S,Embarked_C,Embarked_Q
0,True,False,False
1,False,True,False
2,True,False,False
3,True,False,False
4,True,False,False
5,False,False,True
6,True,False,False
7,True,False,False
8,True,False,False
9,False,True,False


In [28]:
test["Embarked_S"] = (test["Embarked"]=="S")
test["Embarked_C"] = (test["Embarked"]=="C")
test["Embarked_Q"] = (test["Embarked"]=="Q")

## 파생변수 만들기

In [29]:
train["FamilySize"] = train["SibSp"] + train["Parch"] +1    # +1은 "나"를 포함한 것
train["FamilySize"].value_counts()

1     537
2     161
3     102
4      29
6      22
5      15
7      12
11      7
8       6
Name: FamilySize, dtype: int64

In [31]:
train["Family"] = train["FamilySize"]
# 대가족(5 이상): L, 소가족(1):S, 중가족(2~4명): M 으로 표기하려고 한다
train.loc[train["FamilySize"]==1, "Family"] = "S"
train.loc[(train["FamilySize"]>1) & (5>train["FamilySize"]), "Family"] = "M"
train.loc[train["FamilySize"]>4, "Family"] = "L"
train[["Family", "FamilySize"]]

,Family,FamilySize
0,M,2
1,M,2
2,S,1
3,M,2
4,S,1
5,S,1
6,S,1
7,L,5
8,M,3
9,M,2


## 원핫 인코딩

In [32]:
train["Family_S"] = (train["Family"]=="S")
train["Family_M"] = (train["Family"]=="M")
train["Family_L"] = (train["Family"]=="L")
train[["Family_S", "Family_M", "Family_L", "Family", "FamilySize"]]

,Family_S,Family_M,Family_L,Family,FamilySize
0,False,True,False,M,2
1,False,True,False,M,2
2,True,False,False,S,1
3,False,True,False,M,2
4,True,False,False,S,1
5,True,False,False,S,1
6,True,False,False,S,1
7,False,False,True,L,5
8,False,True,False,M,3
9,False,True,False,M,2


In [34]:
test["FamilySize"] = test["SibSp"] + test["Parch"] +1
test["Family"] = test["FamilySize"]
test.loc[test["FamilySize"]==1, "Family"] = "S"
test.loc[(test["FamilySize"]>1) & (5>test["FamilySize"]), "Family"] = "M"
test.loc[test["FamilySize"]>4, "Family"] = "L"
test["Family_S"] = (test["Family"]=="S")
test["Family_M"] = (test["Family"]=="M")
test["Family_L"] = (test["Family"]=="L")
test[["Family_S", "Family_M", "Family_L", "Family", "FamilySize"]]

,Family_S,Family_M,Family_L,Family,FamilySize
0,True,False,False,S,1
1,False,True,False,M,2
2,True,False,False,S,1
3,True,False,False,S,1
4,False,True,False,M,3
5,True,False,False,S,1
6,True,False,False,S,1
7,False,True,False,M,3
8,True,False,False,S,1
9,False,True,False,M,3


## dtypes로 자료형 변경

In [35]:
train["Pclass"].dtypes

dtype('int64')

In [36]:
train["Pclass"] = train["Pclass"].astype("category")

In [37]:
train["Pclass"].dtypes

CategoricalDtype(categories=[1, 2, 3], ordered=False)

# Decision Tree

In [40]:
fn = ["Gender", "Age_mean",     # fn: feature name
      "Embarked_S", "Embarked_C", "Embarked_Q", 
      "Family_S", "Family_M", "Family_L"]
X_train = train[fn]
X_train.head()

,Gender,Age_mean,Embarked_S,Embarked_C,Embarked_Q,Family_S,Family_M,Family_L
0,False,22.0,True,False,False,False,True,False
1,True,38.0,False,True,False,False,True,False
2,True,26.0,True,False,False,True,False,False
3,True,35.0,True,False,False,False,True,False
4,False,35.0,True,False,False,True,False,False


In [41]:
y_label = train["Survived"]

In [43]:
X_test = test[fn]

## 모델링 (Decision Tree)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [64]:
model = DecisionTreeClassifier(max_depth=3)
model

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [65]:
model.fit(X_train, y_label)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [47]:
from sklearn.model_selection import KFold, cross_val_score

In [66]:
kfold = KFold()
kfold

C:\Users\user\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:426: FutureWarning: You should specify a value for 'n_splits' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(NSPLIT_WARNING, FutureWarning)


KFold(n_splits=3, random_state=None, shuffle=False)

In [67]:
kfold = KFold(n_splits=10, shuffle=True, random_state=2019)    # random_state는 random seed와 비슷한 역할
kfold

KFold(n_splits=10, random_state=2019, shuffle=True)

In [68]:
score = cross_val_score(model, X_train, y_label, cv=kfold, scoring="accuracy")    # 출력되는 값은 사실상 정확도를 나타낸다
score

array([0.85555556, 0.80898876, 0.84269663, 0.84269663, 0.80898876,
       0.84269663, 0.74157303, 0.80898876, 0.83146067, 0.79775281])

In [69]:
round(np.mean(score) * 100, 2)    # 정확도 %로

81.81

In [70]:
prediction = model.predict(X_test)
prediction.shape

(418,)

In [71]:
test["Survived"] = prediction
test.columns

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked', 'Age_mean', 'Gender',
       'Embarked_S', 'Embarked_C', 'Embarked_Q', 'FamilySize', 'Family',
       'Family_S', 'Family_M', 'Family_L', 'Survived'],
      dtype='object')

## Kaggle에 Submit

In [74]:
submission = test[["PassengerId", "Survived"]]
submission.to_csv("submit.csv", index=False)

# 스크래핑

urllib: url을 다루는 모듈의 집합

urllib.request: 웹 데이터에 접근

쿠키: 웹서버에 클라이언트가 접근하면 서버쪽에 클라이언트의 정보가 일부 남는데, 그 남겨진 정보를 쿠키라고 한다. 쿠키는 클라이언트 측에 저장하는게 일반적이다.

세션: 서버에 연결할 때 서버에 저장되는 클라이언트의 정보.

In [81]:
import urllib.request

url = "https://www.google.com/images/branding/googlelogo/1x/googlelogo_color_272x92dp.png"
sname = "test.png"
urllib.request.urlretrieve(url, sname)
print("저장됐습니다.")

저장됐습니다.


In [82]:
import urllib.request

url = "https://www.google.com/images/branding/googlelogo/1x/googlelogo_color_272x92dp.png"
sname = "test2.png"
urllib.request.urlopen(url)    # 접속

In [79]:
mem = urllib.request.urlopen(url).read()
mem

b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x10\x00\x00\x00\\\x08\x06\x00\x00\x00\xa6\xe7\xea\xb6\x00\x00\x17\x18IDATx\x01\xed]\x0b\x94\x1cU\x99\xbe:\xd3\x81\xc0.\x82\xc2*\x82\x08\x12\x10\x90\x05\x92\xaa\x9a\x84\x90\xd8]\xb7{\xb2A\xe2A\x81(\xb8\xbb\n\x08\x8a\x1b\\\x84\x98\x05\xe51\x9a\xae\x9a\t\x89\xc0\x02\nA@6\xe1\x81\x06\x17\x10\x1f\x90\xcc$\x01\xf4\x08(\xc8CXX|\x10\x1ed\xfa\x11\x92IUu\x1e\x99dz\xef\xb7\xe6\xb8\xa4\xb7g\xe6\xbf\xd5u\xbb\xaa\x87\xfb\x9dsO\xe71\xd3\xd3g\xea\xd6W\xff\xfd\xff\xef\xff~\x16\x07fv\xad\xdb+\xedTL\xeez\xa7q\xd7\x9fc\xbb\xc1e\xb6\xeb;\xdc\xf1{\xc4\xdf\xe7\x8b\xbf_"\xd6\xb9\xd9\xee`V\xda\xf5\x8fNwUwg\x1a\x1a\x1a\xefL\xa4{6\x1f\xcc\x9d\xcay\xdc\r\xee\xb2]\xff\xcf\xe2\xb5*\xb32yo\xbbx}^|\xef\xcd\xdc\xf5\xff)\xdd\xe5\xed\xcb444\xc6.\xd2\x0b*\x07r7\xf8f&\x1f<\x0b\x12\x88t\xe5\xbd!\xf1\xbe\xabl\'8\xbbs\xd1\xd0\x9eLCCcl\x80\xbb\xdet\xdb\xf1\xef\xcb\xb8\xde\x0e\xdc\xec\xaa\x97 \x92\x01\xee\xfa\x0b\xa7;\xc1\xfe,a\xd0\xd00\x7fxju\xb4\xc5\x18\xd3\xb0\xf3\x1b\xa7\xf

In [ ]:
with open(sname, mode="wb") as f:
    f.write(mem)
    print("이미지가 또 저장됐습니다")

urlopen() 함수는 메모리에 저장

www.kma.go.kr/weather/forecast/mid-term-rss2.jsp

웹 문서 형식: json, html, xml, rdf...
* html이 많이 사용되어왔다.
* html은 비구조 문서 --> 해석이 어렵다.
* xml은 구조화된 문서 --> 기계가 해석 가능. (태그로 묶여져 있으면 기계가 이해하기 쉬워진다)

http://www.weather.go.kr/weather/forecast/mid-term-rss2.jsp?stnId=184

## xml 가지고와서 출력

In [85]:
import urllib.parse
values = {"stnId":"184"}    # 제주도
urllib.parse.urlencode(values)

'stnId=184'

In [90]:
params = urllib.parse.urlencode(values)
API = "http://www.weather.go.kr/weather/forecast/mid-term-rss2.jsp"

url = API + "?" + params

print(url)

http://www.weather.go.kr/weather/forecast/mid-term-rss2.jsp?stnId=184


In [93]:
data = urllib.request.urlopen(url).read()
data

b'<?xml version="1.0" encoding="utf-8" ?>\r\n<rss version="2.0">\r\n<channel>\r\n<title>\xea\xb8\xb0\xec\x83\x81\xec\xb2\xad \xec\x9c\xa1\xec\x83\x81 \xec\xa4\x91\xea\xb8\xb0\xec\x98\x88\xeb\xb3\xb4</title>\r\n<link>http://www.kma.go.kr/weather/forecast/mid-term_07.jsp</link>\r\n<description>\xea\xb8\xb0\xec\x83\x81\xec\xb2\xad \xeb\x82\xa0\xec\x94\xa8 \xec\x9b\xb9\xec\x84\x9c\xeb\xb9\x84\xec\x8a\xa4</description>\r\n<language>ko</language>\r\n<generator>\xea\xb8\xb0\xec\x83\x81\xec\xb2\xad</generator>\r\n<pubDate>2019\xeb\x85\x84 07\xec\x9b\x94 16\xec\x9d\xbc (\xed\x99\x94)\xec\x9a\x94\xec\x9d\xbc 06:00</pubDate>\r\n <item>\r\n<author>\xea\xb8\xb0\xec\x83\x81\xec\xb2\xad</author>\r\n<category>\xec\x9c\xa1\xec\x83\x81\xec\xa4\x91\xea\xb8\xb0\xec\x98\x88\xeb\xb3\xb4</category>\r\n<title>\xec\xa0\x9c\xec\xa3\xbc\xeb\x8f\x84 \xec\x9c\xa1\xec\x83\x81 \xec\xa4\x91\xea\xb8\xb0\xec\x98\x88\xeb\xb3\xb4 - 2019\xeb\x85\x84 07\xec\x9b\x94 16\xec\x9d\xbc (\xed\x99\x94)\xec\x9a\x94\xec\x9d\xbc 06:0

In [94]:
data.decode("utf-8")

'<?xml version="1.0" encoding="utf-8" ?>\r\n<rss version="2.0">\r\n<channel>\r\n<title>기상청 육상 중기예보</title>\r\n<link>http://www.kma.go.kr/weather/forecast/mid-term_07.jsp</link>\r\n<description>기상청 날씨 웹서비스</description>\r\n<language>ko</language>\r\n<generator>기상청</generator>\r\n<pubDate>2019년 07월 16일 (화)요일 06:00</pubDate>\r\n <item>\r\n<author>기상청</author>\r\n<category>육상중기예보</category>\r\n<title>제주도 육상 중기예보 - 2019년 07월 16일 (화)요일 06:00 발표</title>\r\n<link>http://www.kma.go.kr/weather/forecast/mid-term_07.jsp</link>\r\n<guid>http://www.kma.go.kr/weather/forecast/mid-term_07.jsp</guid>\r\n<description>\r\n\t<header>\r\n\t\t<title>제주도 육상중기예보</title>\r\n\t\t<tm>201907160600</tm>\r\n\t\t<wf><![CDATA[19일에 장마전선의 영향으로 비가 오겠고, 그 밖의 날은 고기압 가장자리에 들어 구름이 많겠습니다.<br />기온은 평년(최저기온: 24~25℃, 최고기온: 29~30℃)과 비슷하겠습니다. 강수량은 평년(4~10mm)보다 많겠습니다.<br />바다의 물결은 제주도전해상과 남해서부먼바다에서 19~21일에 1.0~4.0m로 높게 일고, 그 밖의 날은 1.0~2.0m로 일겠습니다.<br /><br />* 열대저압부(TD)의 발달 여부와 북태평양고기압의 확장 정도에 따라 장마전선의 위치와 강수 영역이 달라질 수 있으니, 앞으로 발표

In [95]:
text = data.decode("utf-8")
print(text)

<?xml version="1.0" encoding="utf-8" ?>
<rss version="2.0">
<channel>
<title>기상청 육상 중기예보</title>
<link>http://www.kma.go.kr/weather/forecast/mid-term_07.jsp</link>
<description>기상청 날씨 웹서비스</description>
<language>ko</language>
<generator>기상청</generator>
<pubDate>2019년 07월 16일 (화)요일 06:00</pubDate>
 <item>
<author>기상청</author>
<category>육상중기예보</category>
<title>제주도 육상 중기예보 - 2019년 07월 16일 (화)요일 06:00 발표</title>
<link>http://www.kma.go.kr/weather/forecast/mid-term_07.jsp</link>
<guid>http://www.kma.go.kr/weather/forecast/mid-term_07.jsp</guid>
<description>
	<header>
		<title>제주도 육상중기예보</title>
		<tm>201907160600</tm>
		<wf><![CDATA[19일에 장마전선의 영향으로 비가 오겠고, 그 밖의 날은 고기압 가장자리에 들어 구름이 많겠습니다.<br />기온은 평년(최저기온: 24~25℃, 최고기온: 29~30℃)과 비슷하겠습니다. 강수량은 평년(4~10mm)보다 많겠습니다.<br />바다의 물결은 제주도전해상과 남해서부먼바다에서 19~21일에 1.0~4.0m로 높게 일고, 그 밖의 날은 1.0~2.0m로 일겠습니다.<br /><br />* 열대저압부(TD)의 발달 여부와 북태평양고기압의 확장 정도에 따라 장마전선의 위치와 강수 영역이 달라질 수 있으니, 앞으로 발표되는 기상정보를 참고하기 바랍니다.]]></wf>
	</header>
	<body>
				

		<location 

## 원하는 특정 태그의 내용 추출하기

In [96]:
from bs4 import BeautifulSoup

In [108]:
html = """
<html>
<body>
<h1>멀티캠퍼스</h1>
<h1>역삼동</h1>
<p>1303호</p>
<p>빅데이터 설계반</p>
</body>
</html>
"""

In [110]:
# BeautifulSoup(해석하고자 하는 대상 html문서가 저장된 변수명, 문서를 읽을 때 사용하는 번역기 지정)
soup = BeautifulSoup(html, "html.parser")

In [111]:
soup


<html>
<body>
<h1>멀티캠퍼스</h1>
<h1>역삼동</h1>
<p>1303호</p>
<p>빅데이터 설계반</p>
</body>
</html>

In [112]:
soup.html

<html>
<body>
<h1>멀티캠퍼스</h1>
<h1>역삼동</h1>
<p>1303호</p>
<p>빅데이터 설계반</p>
</body>
</html>

In [113]:
soup.html.body

<body>
<h1>멀티캠퍼스</h1>
<h1>역삼동</h1>
<p>1303호</p>
<p>빅데이터 설계반</p>
</body>

In [114]:
soup.html.body.h1

<h1>멀티캠퍼스</h1>

In [115]:
soup.html.body.h1.string

'멀티캠퍼스'

In [116]:
soup.h1    # 두개의 h1 태그중에 첫번째 것만 출력된다

<h1>멀티캠퍼스</h1>

In [118]:
soup.p

<p>1303호</p>

In [120]:
p1 = soup.html.body.p
p1.next_sibling    # 줄바꿈 문자가 나온다

'\n'

In [121]:
p1.next_sibling.next_sibling

<p>빅데이터 설계반</p>

## id로 요소 추출

In [125]:
html = """
<html><body>
<h1 id="title">스크래핑?</h1>
<p>웹 페이지 분석</p>
<p id="myid">원하는 부분 추출</p>
</body></html>
"""

In [123]:
soup = BeautifulSoup(html, "html5lib")

In [126]:
soup

<html><head></head><body>
<h1 id="title">스크래핑?</h1>
<p>웹 페이지 분석</p>
<p id="myid">원하는 부분 추출</p>

</body></html>

In [127]:
soup.h1

<h1 id="title">스크래핑?</h1>

In [128]:
soup.h1.string

'스크래핑?'

### id로 검색 -- find 함수 (https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find)

In [133]:
title = soup.find(id="title")
title

<h1 id="title">스크래핑?</h1>

In [131]:
myid = soup.find(id="myid")
myid

<p id="myid">원하는 부분 추출</p>

In [132]:
print(title.string)
print(myid.string)

스크래핑?
원하는 부분 추출


In [134]:
soup.find("p")    # find 함수에서는 tag도 찾을 수 있다

<p>웹 페이지 분석</p>

### find_all() 함수

In [135]:
html = """
<html><body>
<ul>
<li><a href="http://www.naver.com">naver</a></li>
<li><a href="http://www.daum.net">daum</a></li>
</ul>
</body></html>
"""

find_all()을 이용한 여러 요소 추출

In [136]:
soup = BeautifulSoup(html, "html.parser")

In [137]:
soup.find_all("ul")

[<ul>
 <li><a href="http://www.naver.com">naver</a></li>
 <li><a href="http://www.daum.net">daum</a></li>
 </ul>]

find_all() 함수는 리스트 안에 여러개가 담겨져 출력된다

In [139]:
soup.find_all("li")    # <li> 태그들이 전부 다 출력된다

[<li><a href="http://www.naver.com">naver</a></li>,
 <li><a href="http://www.daum.net">daum</a></li>]

In [140]:
soup.find_all("a")

[<a href="http://www.naver.com">naver</a>,
 <a href="http://www.daum.net">daum</a>]

In [142]:
links = soup.find_all("a")
for link in links:
    print(link)

<a href="http://www.naver.com">naver</a>
<a href="http://www.daum.net">daum</a>


* a href="http://www.naver.com" 에서 a는 tag, href는 속성, "http://..."는 속성값이라고 한다

In [144]:
for link in links:
    print(link.attrs["href"])

http://www.naver.com
http://www.daum.net


In [145]:
for link in links:
    print(link.string)

naver
daum


In [146]:
for link in links:
    href = link.attrs["href"]
    text = link.string
    print( text, " ===> ", href)

naver  ===>  http://www.naver.com
daum  ===>  http://www.daum.net


# CSS 선택자

css 선택자: jQuery처럼, css 선택자 사용 가능. 특정 요소 추출.

soup.select(<선택자>) <-- 여러개 요소 추출

soup.select_one(<선택자>) <-- 1개 요소 추출

In [152]:
html = """
<html><body>
<h1>역삼동</h1>
<div id="myid">
<h1>멀티캠퍼스</h1>
<ul class="items">
<li>강의실</li>
<li>엘리베이터</li>
<li>식당</li>
</ul>
</div></body></html>
"""

In [153]:
soup = BeautifulSoup(html, "html.parser")
soup


<html><body>
<h1>역삼동</h1>
<div id="myid">
<h1>멀티캠퍼스</h1>
<ul class="items">
<li>강의실</li>
<li>엘리베이터</li>
<li>식당</li>
</ul>
</div></body></html>

In [154]:
soup.select_one("h1")

<h1>역삼동</h1>

In [155]:
soup.select_one("li")

<li>강의실</li>

In [156]:
soup.select("li")

[<li>강의실</li>, <li>엘리베이터</li>, <li>식당</li>]

In [162]:
# "멀티캠퍼스"를 추출하고 싶다
soup.select_one("div#myid")    # id가 myid인 div태그 영역 추출
soup.select_one("div#myid h1")
soup.select_one("div#myid > h1")

<h1>멀티캠퍼스</h1>

In [163]:
soup.select("li")

[<li>강의실</li>, <li>엘리베이터</li>, <li>식당</li>]

In [164]:
soup.select("div#myid ul")

[<ul class="items">
 <li>강의실</li>
 <li>엘리베이터</li>
 <li>식당</li>
 </ul>]

In [165]:
soup.select("div#myid ul.items")    # 클래스명이 items인 ul 영역 추출

[<ul class="items">
 <li>강의실</li>
 <li>엘리베이터</li>
 <li>식당</li>
 </ul>]

In [166]:
soup.select("div#myid ul.items li")

[<li>강의실</li>, <li>엘리베이터</li>, <li>식당</li>]